In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import tensorflow as tf

# define the path to the dataset 
PATH_train = '/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split/train_eq'
PATH_val = '/content/drive/MyDrive/SI_flood_dataset/dataset/SI_flood_dataset_split/val_eq'

# define the batch size
BATCH_SIZE = 32

# define the image size
IMG_SIZE = (256, 256)

# create the train dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory( PATH_train, 
                                                                        batch_size=BATCH_SIZE,
                                                                        image_size=IMG_SIZE,
                                                                        shuffle=True,
                                                                        seed=42,
                                                                        )

# create the validation dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory( PATH_val,
                                                                    batch_size=BATCH_SIZE,
                                                                    image_size=IMG_SIZE,
                                                                    shuffle=True,
                                                                    seed=42,
                                                                    )

# define the label for the classes 
class_names = train_dataset.class_names

# print the class names
print(class_names)

Found 736 files belonging to 2 classes.
Found 186 files belonging to 2 classes.
['flooded', 'non-flooded']


In [3]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [4]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the ResNet block
def resnet_block(inputs, filters, strides=1, activation='relu'):
    x = layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)
    x = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Shortcut connection
    shortcut = inputs
    if strides > 1:
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation(activation)(x)
    return x

# Define the ResNet model
def resnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    x = resnet_block(x, filters=64)
    x = resnet_block(x, filters=64)
    x = resnet_block(x, filters=64)

    x = resnet_block(x, filters=128, strides=2)
    x = resnet_block(x, filters=128)
    x = resnet_block(x, filters=128)

    x = resnet_block(x, filters=256, strides=2)
    x = resnet_block(x, filters=256)
    x = resnet_block(x, filters=256)

    x = resnet_block(x, filters=512, strides=2)
    x = resnet_block(x, filters=512)
    x = resnet_block(x, filters=512)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

# Define the desired image size
image_size = (256, 256)

# Define the number of classes
num_classes = 1

input_shape = (256, 256, 3)

# Create an instance of the ResNet model
model = resnet(input_shape, num_classes)


In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
epochs = 10
train_samples = len(train_dataset)
validation_samples = len(val_dataset)
steps_per_epoch = train_samples // BATCH_SIZE
validation_steps = validation_samples // BATCH_SIZE

hist = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)


Epoch 1/10
23/23 [==============================] - 143s 3s/step - loss: 0.8783 - accuracy: 0.7826 - val_loss: 7787982.0000 - val_accuracy: 0.5000
Epoch 2/10
23/23 [==============================] - 31s 991ms/step - loss: 0.2613 - accuracy: 0.9103 - val_loss: 4739.5776 - val_accuracy: 0.5000
Epoch 3/10
23/23 [==============================] - 37s 1s/step - loss: 0.3439 - accuracy: 0.8899 - val_loss: 1949.4640 - val_accuracy: 0.5000
Epoch 4/10
23/23 [==============================] - 32s 1s/step - loss: 0.2629 - accuracy: 0.8845 - val_loss: 111.0318 - val_accuracy: 0.5000
Epoch 5/10
23/23 [==============================] - 34s 1s/step - loss: 0.2534 - accuracy: 0.8995 - val_loss: 43.9032 - val_accuracy: 0.5000
Epoch 6/10
23/23 [==============================] - 31s 1s/step - loss: 0.2192 - accuracy: 0.9334 - val_loss: 3.7756 - val_accuracy: 0.5054
Epoch 7/10
23/23 [==============================] - 31s 950ms/step - loss: 0.2019 - accuracy: 0.9266 - val_loss: 3.8742 - val_accuracy: 0.532

In [8]:
hist2 = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)


Epoch 1/10
23/23 [==============================] - 33s 1s/step - loss: 0.2595 - accuracy: 0.8927 - val_loss: 2.3625 - val_accuracy: 0.5161
Epoch 2/10
23/23 [==============================] - 31s 1s/step - loss: 0.1822 - accuracy: 0.9361 - val_loss: 2.3784 - val_accuracy: 0.4086
Epoch 3/10
23/23 [==============================] - 31s 971ms/step - loss: 0.1404 - accuracy: 0.9524 - val_loss: 2.0331 - val_accuracy: 0.6935
Epoch 4/10
23/23 [==============================] - 33s 1s/step - loss: 0.1735 - accuracy: 0.9402 - val_loss: 5.3884 - val_accuracy: 0.5323
Epoch 5/10
23/23 [==============================] - 31s 1s/step - loss: 0.1977 - accuracy: 0.9266 - val_loss: 13.0142 - val_accuracy: 0.5376
Epoch 6/10
23/23 [==============================] - 31s 977ms/step - loss: 0.1400 - accuracy: 0.9457 - val_loss: 1.7399 - val_accuracy: 0.8065
Epoch 7/10
23/23 [==============================] - 38s 1s/step - loss: 0.1750 - accuracy: 0.9443 - val_loss: 0.7667 - val_accuracy: 0.7312
Epoch 8/10
23